# Dependencies

In [1]:
# %load_ext autoreload
# %autoreload 2

# For debugging turn on logging to console
import logging
logging.basicConfig(level=logging.DEBUG)
log = logging.getLogger(__name__)

# mongodb
import pymongo as pm

# fine-tuned newspaper lib
from resources.newspaper import newspaper
from resources.newspaper.newspaper.source import Source
from resources.newspaper.newspaper.article import Article

import bs4 as bs
from urllib.parse import urljoin
from dateutil.parser import parse as date_parser
from time import sleep
import random
import pytz
import datetime

import requests

logging.getLogger('urllib3').setLevel(logging.WARNING)

In [ ]:
# for infinite scroll page
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import NoAlertPresentException
import sys

import unittest, time, re

# to divert selenium log stream away
logging.getLogger('selenium').setLevel(logging.WARNING)


# Helpers

In [20]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pm.MongoClient(conn, maxPoolSize=200)

# define db 
# DB_NAME = 'scrape'
DB_NAME = 'FINALP'
db = client[DB_NAME]

def saveToDB(db, collection, url, html, meta={}):
    """
    Saves a document to mongoDB, making sure there are no duplicates by 
    'url' value
    
    Parameters:
    --------
    db, collection  : mongo db connection
    url, html, meta : values to store
    
    Returns:
    --------
    Saved document
    """
    collection = db[collection]
    collection.update_one(
        {'url' : url},
        {
            '$set':
                {'url' : url,
                 'html' : html,
                 'meta' : meta
                }
        }
        ,
        upsert=True
    )
    log.debug(f'Saved to DB')

def scrape(url, db, collection):
    '''
    Scrapes an article from the 'url' up to the 'latest_date'
    
    Parameters:
    --------
    url         : main news website url
    date        : YYYY-MM-DD
    db          : database name
    collection  : mongodb collection
    
    Returns:
    --------
    Article's html and features stored to db, 
    Article's publish date
    
    '''
    log.debug(f"Exctracting features from {url}")
    try:
        article = Article(url)
        article.download()
        # the below method may only extract a snippet... 
        # check the database for results of text extraction
        # and apply additional processing if needed after 
        # article has been stored in the DB
        # see code below Newrepublic for example
        article.parse()
    except Exception as e:
        log.critical(f'Data not saved: {e}')
        return datetime.datetime.now()
    
    saveToDB(db, collection, article.url, article.html, meta={
        'date'    :article.publish_date,
        'title'   :article.title,
        'text'    :article.text,
        'authors' :article.authors
    })
    
    return article.publish_date

def addToDB(DB_NAME,COL_NAME,PATH,FILE):
    '''
    Imports a file into mongoDB
    
    Parameters:
    --------
    DB_NAME : Name of the database to connect to
    COL_NAME: Name of the collection to create
    PATH    : Path to folder with the file
    FILE  : Filename
    
    Returns:
    --------
    Collection COL_NAME in DB_NAME database
    '''
    !mongoimport --db {DB_NAME} --collection {COL_NAME} --file {PATH+FILE} --batchSize 1
    print(f'Collection {COL_NAME} in {DB_NAME} database created')

# NY Post

In [ ]:
collection = 'nypost'
source = 'https://nypost.com/news/'

class Sel(unittest.TestCase):
    def setUp(self):
        self.driver = webdriver.Chrome()
        self.driver.implicitly_wait(30)
        self.base_url = source
        self.verificationErrors = []
        self.accept_next_alert = True
    def getPage(self):
        driver = self.driver
        delay = 2
        driver.get(self.base_url)
        html_source = driver.page_source
        self.html = html_source.encode('utf-8')
        return self.html
    def scrollDown(self):
        driver = self.driver
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        self.html = driver.page_source.encode('utf-8')
        return driver.page_source 
    def shutdown(self):
        driver = self.driver
        driver.quit()

data = Sel()

data.setUp()
data.getPage()
page = 1

utc=pytz.UTC
earliest_date = utc.localize(date_parser('2017-01-01'))

scraped_urls = []

while True:
    log.debug(f'NEXT SCROLL (#{page})')
    page += 1
    
    html = data.scrollDown()
    soup = bs.BeautifulSoup(html,'lxml')

    for link in soup.find('div', {"id":"primary"}).findAll('a'):
        url = urljoin(source, link['href'])
        if url and url not in scraped_urls:
            scraped_urls.append(url)
            log.debug(f'Processing url: {url}')
            article_date = scrape(url, db, collection)
    try:
        if article_date < earliest_date:
            log.debug(f'Reached earliest date requested: {article_date}')
            break
    except Exception as e:
        log.debug(f"Something is wrong: {e}")
data.shutdown()

In [ ]:
collection = 'nypost'
source = 'https://nypost.com/news/page/'
page   = 182

utc=pytz.UTC
earliest_date = date_parser('2017-01-01')

while True:
    log.debug(f'\n\n PROCESSING PAGE: {source+str(page)}\n\n\
              ====================================\n\n')
    s = Source(source+str(page))
    page += 1
    s.download()
    soup = bs.BeautifulSoup(s.html,'lxml')

    for h3 in soup.find('div', {"id":"primary"}).findAll('h3'):
        for link in h3.findAll('a'):
            url = link['href']
            log.debug(f'Processing url: {url}')

            try:
                article_date = scrape(url, db, collection)
            except Exception as e:
                log.debug(e)
                article_date = earliest_date + 1 #to make sure scraping continues

        try:
            if article_date < earliest_date:
                log.debug(f'Reached earliest date requested: {article_date}')
                break
        except Exception as e:
            log.debug(f'Exception: {e}')
            continue

DEBUG:__main__:

 PROCESSING PAGE: https://nypost.com/news/page/182



DEBUG:__main__:Processing url: https://nypost.com/2018/06/20/giuliani-tells-trump-campaign-manager-to-keep-quiet-after-sessions-rant/
DEBUG:__main__:Exctracting features from https://nypost.com/2018/06/20/giuliani-tells-trump-campaign-manager-to-keep-quiet-after-sessions-rant/
DEBUG:__main__:Saved to DB
DEBUG:__main__:Processing url: https://nypost.com/2018/06/20/man-arrested-in-london-subway-explosion/
DEBUG:__main__:Exctracting features from https://nypost.com/2018/06/20/man-arrested-in-london-subway-explosion/
DEBUG:__main__:Saved to DB
DEBUG:__main__:Processing url: https://nypost.com/2018/06/20/indonesia-says-180-could-be-dead-after-tourist-boat-sinks/
DEBUG:__main__:Exctracting features from https://nypost.com/2018/06/20/indonesia-says-180-could-be-dead-after-tourist-boat-sinks/
DEBUG:__main__:Saved to DB
DEBUG:__main__:Processing url: https://nypost.com/2018/06/20/mom-says-principal-humiliated-12-year-old-dau

DEBUG:__main__:Saved to DB
DEBUG:__main__:Processing url: https://nypost.com/2018/06/19/teen-plotted-isis-style-school-bombing-to-cause-fear-cops-say/
DEBUG:__main__:Exctracting features from https://nypost.com/2018/06/19/teen-plotted-isis-style-school-bombing-to-cause-fear-cops-say/
DEBUG:__main__:Saved to DB
DEBUG:__main__:Processing url: https://nypost.com/2018/06/19/white-house-deputy-chief-of-staff-to-resign/
DEBUG:__main__:Exctracting features from https://nypost.com/2018/06/19/white-house-deputy-chief-of-staff-to-resign/
DEBUG:__main__:Saved to DB
DEBUG:__main__:

 PROCESSING PAGE: https://nypost.com/news/page/185



DEBUG:__main__:Processing url: https://nypost.com/2018/06/19/trump-administration-giving-nonprofit-nearly-460m-to-house-migrant-kids/
DEBUG:__main__:Exctracting features from https://nypost.com/2018/06/19/trump-administration-giving-nonprofit-nearly-460m-to-house-migrant-kids/
DEBUG:__main__:Saved to DB
DEBUG:__main__:Processing url: https://nypost.com/2018/06/19/un

DEBUG:__main__:Exctracting features from https://nypost.com/2018/06/19/dennis-rodman-heaps-praise-upon-friend-for-life-kim-jong-un/
DEBUG:__main__:Saved to DB
DEBUG:__main__:Processing url: https://nypost.com/2018/06/19/horny-couple-joins-mile-high-club-in-full-view-of-passengers/
DEBUG:__main__:Exctracting features from https://nypost.com/2018/06/19/horny-couple-joins-mile-high-club-in-full-view-of-passengers/
DEBUG:__main__:Saved to DB
DEBUG:__main__:Processing url: https://nypost.com/2018/06/19/white-house-downplays-china-trade-tensions-as-dispute/
DEBUG:__main__:Exctracting features from https://nypost.com/2018/06/19/white-house-downplays-china-trade-tensions-as-dispute/
DEBUG:__main__:Saved to DB
DEBUG:__main__:Processing url: https://nypost.com/2018/06/19/inspector-general-fbi-didnt-let-anti-trump-views-affect-probe/
DEBUG:__main__:Exctracting features from https://nypost.com/2018/06/19/inspector-general-fbi-didnt-let-anti-trump-views-affect-probe/
DEBUG:__main__:Saved to DB
DEBU

DEBUG:__main__:Saved to DB
DEBUG:__main__:Processing url: https://nypost.com/2018/06/19/army-officially-splits-with-west-point-commie-cadet/
DEBUG:__main__:Exctracting features from https://nypost.com/2018/06/19/army-officially-splits-with-west-point-commie-cadet/
DEBUG:__main__:Saved to DB
DEBUG:__main__:Processing url: https://nypost.com/2018/06/19/ncaa-basketball-referee-pleads-guilty-to-molesting-teen-boy/
DEBUG:__main__:Exctracting features from https://nypost.com/2018/06/19/ncaa-basketball-referee-pleads-guilty-to-molesting-teen-boy/
DEBUG:__main__:Saved to DB
DEBUG:__main__:Processing url: https://nypost.com/2018/06/19/china-blasts-trumps-new-tariff-threat-as-blackmail/
DEBUG:__main__:Exctracting features from https://nypost.com/2018/06/19/china-blasts-trumps-new-tariff-threat-as-blackmail/
DEBUG:__main__:Saved to DB
DEBUG:__main__:Processing url: https://nypost.com/2018/06/18/china-says-kim-jong-un-is-planning-a-visit-to-beijing/
DEBUG:__main__:Exctracting features from https:/

DEBUG:__main__:Exctracting features from https://nypost.com/2018/06/18/michelle-obama-joins-laura-bush-to-oppose-trumps-immigration-policy/
DEBUG:__main__:Saved to DB
DEBUG:__main__:Processing url: https://nypost.com/2018/06/18/sisters-accused-of-stabbing-each-other-in-mutual-combat/
DEBUG:__main__:Exctracting features from https://nypost.com/2018/06/18/sisters-accused-of-stabbing-each-other-in-mutual-combat/
DEBUG:__main__:Saved to DB
DEBUG:__main__:Processing url: https://nypost.com/2018/06/18/no-a-werewolf-was-not-killed-in-montana/
DEBUG:__main__:Exctracting features from https://nypost.com/2018/06/18/no-a-werewolf-was-not-killed-in-montana/
DEBUG:__main__:Saved to DB
DEBUG:__main__:Processing url: https://nypost.com/2018/06/18/teacher-nearly-struck-by-lightning-while-recording-incoming-storm/
DEBUG:__main__:Exctracting features from https://nypost.com/2018/06/18/teacher-nearly-struck-by-lightning-while-recording-incoming-storm/
DEBUG:__main__:Saved to DB
DEBUG:__main__:Processing 

...

DEBUG:__main__:Saved to DB
DEBUG:__main__:Processing url: https://nypost.com/2018/05/25/bones-found-in-shipwreck-dont-belong-to-infamous-pirate/
DEBUG:__main__:Exctracting features from https://nypost.com/2018/05/25/bones-found-in-shipwreck-dont-belong-to-infamous-pirate/
DEBUG:__main__:Saved to DB
DEBUG:__main__:Processing url: https://nypost.com/2018/05/25/court-rules-inmate-serving-life-cant-sue-because-hes-dead/
DEBUG:__main__:Exctracting features from https://nypost.com/2018/05/25/court-rules-inmate-serving-life-cant-sue-because-hes-dead/
DEBUG:__main__:Saved to DB
DEBUG:__main__:Processing url: https://nypost.com/2018/05/25/explosion-at-toronto-restaurant-injures-15-people/
DEBUG:__main__:Exctracting features from https://nypost.com/2018/05/25/explosion-at-toronto-restaurant-injures-15-people/
DEBUG:__main__:Saved to DB
DEBUG:__main__:Processing url: https://nypost.com/2018/05/25/lava-from-hawaii-volcano-now-entering-pacific-from-third-flow/
DEBUG:__main__:Exctracting features fr

DEBUG:__main__:Exctracting features from https://nypost.com/2018/05/24/george-zimmerman-is-broke/
DEBUG:__main__:Saved to DB
DEBUG:__main__:

 PROCESSING PAGE: https://nypost.com/news/page/301



DEBUG:__main__:Processing url: https://nypost.com/2018/05/24/stormy-daniels-again-wants-out-of-non-disclosure-agreement/
DEBUG:__main__:Exctracting features from https://nypost.com/2018/05/24/stormy-daniels-again-wants-out-of-non-disclosure-agreement/
DEBUG:__main__:Saved to DB
DEBUG:__main__:Processing url: https://nypost.com/2018/05/24/sex-offender-caught-in-bed-with-teens-by-roommate-cops/
DEBUG:__main__:Exctracting features from https://nypost.com/2018/05/24/sex-offender-caught-in-bed-with-teens-by-roommate-cops/
DEBUG:__main__:Saved to DB
DEBUG:__main__:Processing url: https://nypost.com/2018/05/24/deadbeat-son-who-was-sued-by-parents-has-one-week-to-move-out/
DEBUG:__main__:Exctracting features from https://nypost.com/2018/05/24/deadbeat-son-who-was-sued-by-parents-has-one-week-to-move-o

DEBUG:__main__:Processing url: https://nypost.com/2018/05/23/remains-found-in-drainage-pipe-idd-as-teen-who-vanished-over-2-years-ago/
DEBUG:__main__:Exctracting features from https://nypost.com/2018/05/23/remains-found-in-drainage-pipe-idd-as-teen-who-vanished-over-2-years-ago/
DEBUG:__main__:Saved to DB
DEBUG:__main__:Processing url: https://nypost.com/2018/05/23/hawaii-volcanos-latest-phenomenon-blue-flames/
DEBUG:__main__:Exctracting features from https://nypost.com/2018/05/23/hawaii-volcanos-latest-phenomenon-blue-flames/
DEBUG:__main__:Saved to DB
DEBUG:__main__:

 PROCESSING PAGE: https://nypost.com/news/page/304



DEBUG:__main__:Processing url: https://nypost.com/2018/05/23/nearly-100-illegal-immigrants-found-in-trailer-during-traffic-stop/
DEBUG:__main__:Exctracting features from https://nypost.com/2018/05/23/nearly-100-illegal-immigrants-found-in-trailer-during-traffic-stop/
DEBUG:__main__:Saved to DB
DEBUG:__main__:Processing url: https://nypost.com/2018/05/23/makeup-artist

DEBUG:__main__:Saved to DB
DEBUG:__main__:Processing url: https://nypost.com/2018/05/23/alleged-gay-village-serial-killer-appears-in-court/
DEBUG:__main__:Exctracting features from https://nypost.com/2018/05/23/alleged-gay-village-serial-killer-appears-in-court/
DEBUG:__main__:Saved to DB
DEBUG:__main__:Processing url: https://nypost.com/2018/05/23/schumer-blames-trump-for-soaring-gas-prices/
DEBUG:__main__:Exctracting features from https://nypost.com/2018/05/23/schumer-blames-trump-for-soaring-gas-prices/
DEBUG:__main__:Saved to DB
DEBUG:__main__:Processing url: https://nypost.com/2018/05/23/trump-lambastes-ms-13-refers-to-them-as-animals-again/
DEBUG:__main__:Exctracting features from https://nypost.com/2018/05/23/trump-lambastes-ms-13-refers-to-them-as-animals-again/
DEBUG:__main__:Saved to DB
DEBUG:__main__:Processing url: https://nypost.com/2018/05/23/jared-kushner-finally-gets-permanent-security-clearance/
DEBUG:__main__:Exctracting features from https://nypost.com/2018/05/23/jar

DEBUG:__main__:Processing url: https://nypost.com/2018/05/23/ms-13-member-sentenced-to-40-years-in-prison-for-killing-teen-boy/
DEBUG:__main__:Exctracting features from https://nypost.com/2018/05/23/ms-13-member-sentenced-to-40-years-in-prison-for-killing-teen-boy/
DEBUG:__main__:Saved to DB
DEBUG:__main__:Processing url: https://nypost.com/2018/05/23/thousands-of-vegas-casino-workers-ok-strike-that-could-leave-resorts-reeling/
DEBUG:__main__:Exctracting features from https://nypost.com/2018/05/23/thousands-of-vegas-casino-workers-ok-strike-that-could-leave-resorts-reeling/
DEBUG:__main__:Saved to DB
DEBUG:__main__:Processing url: https://nypost.com/2018/05/23/recent-inspection-uncovers-violations-at-water-park-where-boy-was-decapitated/
DEBUG:__main__:Exctracting features from https://nypost.com/2018/05/23/recent-inspection-uncovers-violations-at-water-park-where-boy-was-decapitated/
DEBUG:__main__:Saved to DB
DEBUG:__main__:Processing url: https://nypost.com/2018/05/23/teen-charged-i

In [25]:
FIX AUTHORS

True